As a stepping stone between SATLAS and SATLAS2, an interface has been provided which can mostly be used as a drop-in replacement for code that uses the SATLAS syntax. Note that not all functionalities have been implemented in this fashion. For users that require these functionalities, we recommend migrating to SATLAS2.

In [1]:
import sys
import time

import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
import numpy as np

sys.path.insert(0, '..\src')

import satlas2
import satlas as sat


def modifiedSqrt(input):
    output = np.sqrt(input)
    output[input <= 0] = 1e-3
    return output

# Fitting a single hyperfine spectrum

The most common task, and the one this interface is meant for, is fitting a single hyperfine spectrum. A special class in SATLAS2 called *HFSModel* has been created as a replacement for the equivalent SATLAS *HFSModel*. Note that the normal hyperfine spectrum model in SATLAS2 is called *HFS*.

In [2]:
spin = 3.5
J = [0.5, 1.5]
A = [9600, 175]
B = [0, 315]
C = [0, 0]
FWHMG = 135
FWHML = 101
centroid = 480
bkg = [100]
scale = 90

x = np.arange(-17500, -14500, 40)
x = np.hstack([x, np.arange(20000, 23000, 40)])

rng = np.random.default_rng(0)
hfs = satlas2.HFSModel(I=spin,
                       J=J,
                       ABC=[A[0], A[1], B[0], B[1], C[0], C[1]],
                       centroid=centroid,
                       fwhm=[FWHMG, FWHML],
                       scale=scale,
                       background_params=bkg,
                       use_racah=True)
hfs.set_variation({'Cu': False})

C:\Users\megat\Documents\git_repositories\personal\satlas2\examples\..\src\satlas2\models\hfsModel.py:284: RuntimeWarning: invalid value encountered in scalar divide
  shift = phase * n / d


AttributeError: 'HFSModel' object has no attribute 'set_variation'

The object called *hfs* can be used with the syntax of SATLAS. Generating Poisson-distributed data is done by simply calling the function with frequency values as an argument, and using the result for the NumPy Poisson random number generator.

In [ ]:
y = hfs(x)
y = rng.poisson(y)

In order to demonstrate the difference in performance, the centroid is offset by 100 from the actual value and the fitting is done by both the interface and SATLAS.

In [ ]:
hfs.params['centroid'].value = centroid - 100
hfs1 = sat.HFSModel(spin,
                    J, [A[0], A[1], B[0], B[1], C[0], C[1]],
                    centroid - 100, [FWHMG, FWHML],
                    scale=scale,
                    background_params=bkg,
                    use_racah=True)
hfs1.set_variation({'Cu': False})

print('Fitting 1 dataset with chisquare (Pearson, satlas2)...')
start = time.time()
f = satlas2.chisquare_fit(hfs, x, y, modifiedSqrt(y), show_correl=False)
stop = time.time()
dt1 = stop - start

print('Fitting 1 dataset with chisquare (Pearson, satlas1)...')
start = time.time()
sat.chisquare_fit(hfs1, x, y, modifiedSqrt(y))
hfs1.display_chisquare_fit(show_correl=False)
stop = time.time()
dt2 = stop - start
print('SATLAS2: {:.3} s, {:.0f} function evaluations'.format(
    dt1, f.result.nfev))
print('SATLAS1: {:.3} s'.format(dt2))

fig = plt.figure(constrained_layout=True, figsize=(14, 9))
gs = gridspec.GridSpec(nrows=2, ncols=2, figure=fig)
ax11 = fig.add_subplot(gs[0, 0])
ax11.label_outer()
ax12 = fig.add_subplot(gs[0, 1])
ax12.label_outer()
ax21 = fig.add_subplot(gs[1, 0])
ax21.label_outer()
ax22 = fig.add_subplot(gs[1, 1])
ax22.label_outer()

ax11.errorbar(x, y, modifiedSqrt(y), fmt='.', label='Artificial data')
ax11.plot(x, hfs(x), '-', label='SATLAS2 fit')
ax11.set_xlim(-17500, -14500)
ax12.errorbar(x, y, modifiedSqrt(y), fmt='.', label='Artificial data')
ax12.plot(x, hfs(x), '-', label='SATLAS2 fit')
ax12.set_xlim(20000, 23000)
ax21.errorbar(x, y, modifiedSqrt(y), fmt='.', label='Artificial data')
ax21.plot(x, hfs1(x), '-', label='SATLAS fit')
ax21.set_xlim(-17500, -14500)
ax22.errorbar(x, y, modifiedSqrt(y), fmt='.', label='Artificial data')
ax22.plot(x, hfs1(x), '-', label='SATLAS fit')
ax22.set_xlim(20000, 23000)
ax11.legend()
ax21.legend()
plt.show()

Note that the results are functionally identical: the slight difference is due to a more modern implementation of the least squares fitting routine that is used under the hood by SATLAS2. The speedup by using SATLAS 2 is about a factor 20 for a single spectrum.

# Overlapping hyperfine spectra

The other most common usecase for SATLAS was analysis of spectra with an isomer present, resulting in overlapping spectra. In the SATLAS terminology, this would result in a *SumModel* being used. In SATLAS2, a second *HFS* is simply added to the Source. However, the interface does provide the folllowing functionality:

In [ ]:
J = [0.5, 1.5]
FWHMG = 135
FWHML = 101

spin1 = 4
A1 = [5300, 100]
B1 = [0, 230]
C1 = [0, 0]
centroid1 = 400
bkg1 = 60
scale1 = 90

spin2 = 7
A2 = [3300, 60]
B2 = [0, 270]
C2 = [0, 0]
centroid2 = -100
bkg2 = 60
scale2 = 160

x = np.arange(-13000, -9000, 40)
x = np.hstack([x, np.arange(11000, 14000, 40)])
rng = np.random.default_rng(0)

hfs1 = satlas2.HFSModel(I=spin1,
                        J=J,
                        ABC=[A1[0], A1[1], B1[0], B1[1], C1[0], C1[1]],
                        centroid=centroid1,
                        fwhm=[FWHMG, FWHML],
                        scale=scale1,
                        background_params=[bkg1],
                        use_racah=True)
hfs1.set_variation({'Cu': False})
hfs2 = satlas2.HFSModel(I=spin2,
                        J=J,
                        ABC=[A2[0], A2[1], B2[0], B2[1], C2[0], C2[1]],
                        centroid=centroid2,
                        fwhm=[FWHMG, FWHML],
                        scale=scale2,
                        background_params=[bkg2],
                        use_racah=True)
hfs2.set_variation({'Cu': False})
y = hfs1.f(x) + hfs2.f(x) + satlas2.Step([bkg1, bkg2], [0]).f(x)
y = rng.poisson(y)

hfs1.params['centroid'].value = centroid1 - 100
hfs2.params['centroid'].value = centroid2 - 100
summodel = satlas2.SumModel([hfs1, hfs2], {
    'values': [bkg1, bkg2],
    'bounds': [0]
})

hfs3 = sat.HFSModel(spin1,
                    J, [A1[0], A1[1], B1[0], B1[1], C1[0], C1[1]],
                    centroid - 100, [FWHMG, FWHML],
                    scale=scale1,
                    background_params=bkg,
                    use_racah=True)
hfs4 = sat.HFSModel(spin2,
                    J, [A2[0], A2[1], B2[0], B2[1], C2[0], C2[1]],
                    centroid - 100, [FWHMG, FWHML],
                    scale=scale2,
                    background_params=[0],
                    use_racah=True)
hfs3.set_variation({'Cu': False})
hfs4.set_variation({'Background0': False, 'Cu': False})
summodel2 = hfs3 + hfs4

print('Fitting 1 dataset with chisquare (Pearson, satlas2)...')
start = time.time()
f = satlas2.chisquare_fit(summodel, x, y, modifiedSqrt(y), show_correl=False)
stop = time.time()
dt1 = stop - start
start = time.time()
sat.chisquare_fit(summodel2, x, y, modifiedSqrt(y))
summodel2.display_chisquare_fit(show_correl=False)
stop = time.time()
dt2 = stop - start
print('SATLAS2: {:.3} s, {:.0f} function evaluations'.format(
    dt1, f.result.nfev))
print('SATLAS1: {:.3} s'.format(dt2, f.result.nfev))

fig = plt.figure(constrained_layout=True, figsize=(14, 9))
gs = gridspec.GridSpec(nrows=2, ncols=2, figure=fig)
ax11 = fig.add_subplot(gs[0, 0])
ax11.label_outer()
ax12 = fig.add_subplot(gs[0, 1])
ax12.label_outer()
ax21 = fig.add_subplot(gs[1, 0])
ax21.label_outer()
ax22 = fig.add_subplot(gs[1, 1])
ax22.label_outer()

ax11.errorbar(x, y, modifiedSqrt(y), fmt='.', label='Artificial data')
ax11.plot(x, hfs1.f(x), '-', label='SATLAS2 fit model 1')
ax11.plot(x, hfs2.f(x), '-', label='SATLAS2 fit model 2')
ax11.plot(x, summodel.f(x), '-', label='Sum of models')
ax11.set_xlim(-13000, -9000)
ax12.errorbar(x, y, modifiedSqrt(y), fmt='.', label='Artificial data')
ax12.plot(x, hfs1.f(x), '-', label='SATLAS2 fit model 1')
ax12.plot(x, hfs2.f(x), '-', label='SATLAS2 fit model 2')
ax12.plot(x, summodel.f(x), '-', label='Sum of models')
ax12.set_xlim(11000, 14000)
ax11.legend()

ax21.errorbar(x, y, modifiedSqrt(y), fmt='.', label='Artificial data')
ax21.plot(x, hfs3(x), '-', label='SATLAS fit model 1')
ax21.plot(x, hfs4(x), '-', label='SATLAS fit model 2')
ax21.plot(x, summodel2(x), '-', label='Sum of models')
ax21.set_xlim(-13000, -9000)
ax22.errorbar(x, y, modifiedSqrt(y), fmt='.', label='Artificial data')
ax22.plot(x, hfs3(x), '-', label='SATLAS fit model 1')
ax22.plot(x, hfs4(x), '-', label='SATLAS fit model 2')
ax22.plot(x, summodel2(x), '-', label='Sum of models')
ax22.set_xlim(11000, 14000)
ax21.legend()
plt.show()

The difference in result is due to the original SATLAS not implementing a Step background model, where the constant background has a different value left and right of some user-provided threshold value. Notice here that the speedup bue using the SATLAS2 implementation has risen from a factor 20 for a single spectrum to a factor 60.